# Documentation Coming Soon

In [13]:
import javaobj
import os

In [14]:
root_path = '.'
output_folder = 'outputs'

In [20]:
def load_ser(path):
    with open(path, "rb") as fd:
        jobj = fd.read()
    return javaobj.loads(jobj)

In [21]:
def add_to_list(file_name,objects):
    file_names.append(file_name)
    print(f'Loading {item}')
    objects.append(load_ser(f'{root_path}/{output_folder}/{item}'))

In [19]:
file_names = []
objects = []
for item in os.listdir(f'{root_path}/{output_folder}'):
    file_names.append(item)
    print(f'Loading {item}')
    objects.append(load_ser(f'{root_path}/{output_folder}/{item}'))

Loading pranjal_output_small_120411.ser


### Helper Functions

In [23]:
def command_are_equal(a,b):
    return a.type==b.type and a.val == b.val and a.output == b.output and a.corr==b.corr and a.corr_addr == b.corr_addr and a.allocSize==b.allocSize and a.freeSize==b.freeSize


def compareTwo(obj1,obj2):
    if(len(obj1.testCases)!=len(obj2.testCases)):
        return False
    t = len(obj1.testCases)
    for i in range(t):
        comm1 = obj1.testCases[i].commands
        comm2 = obj2.testCases[i].commands
        if(len(comm1)!=len(comm2)):
            print(i)
            print(len(comm1))
            print(len(comm2))
            return False
        n_comm = len(comm1)
        
        for j in range(n_comm):
            if(not command_are_equal(comm1[j],comm2[j])):
                print(i,j,comm1[j].type,comm2[j].type,comm1[j].output,comm2[j].output)
                return i,j,comm1[j],comm2[j]
    return True

def is_in_comm_ops(arr,i):
    for item in arr:
        if(i in item):
            return True
    return False

# Compare all outputs

In [24]:
comm_ops = [[]]
row_num  = -1
for k in range(len(objects)):
    if(is_in_comm_ops(comm_ops,k)):
        continue
    row_num = row_num+1
    if(row_num>0):
        comm_ops.append([])
    comm_ops[row_num] = [k]
    for i in range(k+1,len(objects),1):
        print(k,i)
        outp = compareTwo(objects[k],objects[i])
        if(outp == True):
            comm_ops[row_num].append(i)
            

# PRINT SAME OUTPUTS

In [25]:
for i,item in enumerate(comm_ops):
    print(f'Same Outp {i+1}: {[file_names[x].replace(".ser","").replace("_output","") for x in item]}')

Same Outp 1: ['pranjal_small_120411']


## Sanity Check

In [26]:
for i,item in enumerate(objects):
    item = objects[i]
    tests = item.testCases
    for j,t in enumerate(tests):
        commands = t.commands
        for k,comm in enumerate(commands):
            if(comm.type=='Sanity'):
                if(comm.output!=0):
                    print(f'Sanity Error in {file_names[i]}:- Test Case {i} Command Num {k}. Value:- {comm.output}')

Enjoy :)

In [29]:
# Total Time taken in Defragment
objects[0].time_taken[1].value

53208100.0

In [31]:
# Compare where you went wrong

diff = compareTwo(objects[0],objects[0])